In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict, RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier, \
RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import itertools as it
import time as time
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

import warnings
from IPython import display
import re

In [14]:
train = pd.read_csv('train_classification.csv')
test = pd.read_csv('test_classification.csv')
train.head()
# train.columns
h_id = test.host_id
test.head()

id     host_id  host_since host_location  \
0  1543972437713169913  1521551465  2020-12-16           NaN   
1  1710552057351883447   429737865  2016-12-16  New York, NY   
2             97075525   995041819  2019-03-14           NaN   
3             83734823    97552339  2014-12-02   Chicago, IL   
4             56722823  1029857633  2019-04-22           NaN   

  host_response_time host_response_rate host_acceptance_rate  \
0       within a day                88%                  80%   
1     within an hour               100%                  97%   
2     within an hour                99%                  99%   
3     within an hour               100%                 100%   
4       within a day               100%                  65%   

    host_neighbourhood  host_listings_count  host_total_listings_count  \
0       Near West Side                   44                         44   
1            Cambridge                 4820                       6438   
2  West Loop/Greektown                   71                         88   
3        Streeterville                   14                         47   
4          River North                   20                         22   

                 host_verifications host_has_profile_pic  \
0                ['email', 'phone']                    t   
1  ['email', 'phone', 'work_email']                    t   
2                         ['phone']                    t   
3                ['email', 'phone']                    t   
4                ['email', 'phone']                    t   

  host_identity_verified neighbourhood_cleansed   latitude  longitude  \
0                      t           Lincoln Park  41.931102 -87.667468   
1                      t        Near North Side  41.891162 -87.622263   
2                      t        Near North Side  41.895350 -87.626040   
3                      t        Near North Side  41.891376 -87.620269   
4                      t        Near North Side  41.894760 -87.625640   

        property_type        room_type  accommodates bathrooms_text  beds  \
0  Entire rental unit  Entire home/apt            14         1 bath   1.0   
1  Entire rental unit  Entire home/apt             2         1 bath   1.0   
2  Entire rental unit  Entire home/apt             8        2 baths   3.0   
3  Entire rental unit  Entire home/apt             4         1 bath   1.0   
4        Entire condo  Entire home/apt             3         1 bath   1.0   

   minimum_nights  maximum_nights  minimum_minimum_nights  \
0              32             730                      32   
1              32            1125                      32   
2               3             344                       2   
3              32            1125                      32   
4              32             365                      32   

   maximum_minimum_nights  minimum_maximum_nights  maximum_maximum_nights  \
0                      32                     730                     730   
1                     366                    1125                    1125   
2                       3                    1125                    1125   
3                      32                    1125                    1125   
4                      32                     365                     365   

   minimum_nights_avg_ntm  maximum_nights_avg_ntm has_availability  \
0                    32.0                   730.0                t   
1                   354.2                  1125.0                t   
2                     2.2                  1125.0                t   
3                    32.0                  1125.0                t   
4                    32.0                   365.0                t   

   availability_30  availability_60  availability_90  availability_365  \
0               30               60               90               365   
1                0                1               31               306   
2               20               47               77               34

In [15]:
pd.set_option('display.max_columns', None)

# train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)

train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
train['host_since'] = pd.to_datetime(train['host_since'])

train['host_is_superhost'] = train['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

train['instant_bookable'] = train['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
train['host_has_profile_pic'] = train['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
train['host_identity_verified'] = train['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
train['has_availability'] = train['has_availability'].apply(lambda x: 1 if x == 't' else 0)


'------------------------------------------------------------------------------------------------'

test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_since'] = pd.to_datetime(test['host_since'])

test['instant_bookable'] = test['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
test['host_has_profile_pic'] = test['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
test['host_identity_verified'] = test['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
test['has_availability'] = test['has_availability'].apply(lambda x: 1 if x == 't' else 0)


# test['host_is_superhost'] = test['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)


# covert host_since into a usuable form
current_date = pd.to_datetime("today")

train['days_since_host_started'] = (current_date - train['host_since']).dt.days
test['days_since_host_started'] = (current_date - test['host_since']).dt.days

# Impute missing values

#Impute numerical columns with the mean
for column in train.select_dtypes(include=['number']).columns:
    train[column].fillna(train[column].mean(), inplace=True)
    
for column in test.select_dtypes(include=['number']).columns:
    test[column].fillna(train[column].mean(), inplace=True)  # Use train's mean to avoid data leakage

# Impute categorical columns with the mode (most frequent value)
for column in train.select_dtypes(include=['object', 'category']).columns:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
for column in test.select_dtypes(include=['object', 'category']).columns:
    test[column].fillna(train[column].mode()[0], inplace=True)  # Use train's mode to avoid data leakage


# print(test.isnull().any())
# print('---------------------------------------------------------')
# print(train.isnull().any())

def extract_bathrooms(text):
    if 'half' in text.lower():
        return 0.5
    else:
        match = re.search(r'\d+\.?\d*', text)
        return float(match.group()) if match else np.nan

train['bathrooms'] = train['bathrooms_text'].apply(extract_bathrooms)
test['bathrooms'] = test['bathrooms_text'].apply(extract_bathrooms)

train['bathrooms'].fillna(train['bathrooms'].mean(), inplace=True)
test['bathrooms'].fillna(test['bathrooms'].mean(), inplace=True)


train_categories = set(train['host_location'].unique())
test['host_location'] = test['host_location'].apply(lambda x: x if x in train_categories else np.nan)
test['host_location'].fillna(train['host_location'].mode()[0], inplace=True)

train['host_verifications_count'] = train['host_verifications'].apply(lambda x: len(x.split(',')))
test['host_verifications_count'] = test['host_verifications'].apply(lambda x: len(x.split(',')))

train.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)
test.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)

train = pd.get_dummies(train)
test = pd.get_dummies(test)




train.head()

id  host_response_rate  host_acceptance_rate  \
0            107745711                96.0                  95.0   
1             78539203               100.0                  97.0   
2  1112961466060884803               100.0                 100.0   
3            104794437               100.0                  98.0   
4  1491898404543717797               100.0                 100.0   

   host_is_superhost  host_listings_count  host_total_listings_count  \
0                  0                   11                         13   
1                  0                 1141                       2346   
2                  0                    1                          1   
3                  0                   55                         58   
4                  1                   74                         75   

   host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                     1                       1  41.794240 -87.661380   
1                     1                       1  41.892749 -87.620711   
2                     1                       1  41.948160 -87.652309   
3                     1                       1  41.801030 -87.597160   
4                     1                       1  41.855770 -87.624130   

   accommodates  beds  minimum_nights  maximum_nights  minimum_minimum_nights  \
0             1   1.0              32            1125                      32   
1            12   3.0              32             365                      32   
2             6   3.0               2              45                       2   
3             2   1.0               2             180                       2   
4             6   3.0               2             365                       2   

   maximum_minimum_nights  minimum_maximum_nights  maximum_maximum_nights  \
0                      32                    1125                    1125   
1                      32                     365                     365   
2                       2                      45                      45   
3                       2                     180                     180   
4                       2                    1125                    1125   

   minimum_nights_avg_ntm  maximum_nights_avg_ntm  has_availability  \
0                    32.0                  1125.0                 1   
1                    32.0                   365.0                 1   
2                     2.0                    45.0                 1   
3                     2.0                   180.0                 1   
4                     2.0                  1125.0                 1   

   availability_30  availability_60  availability_90  availability_365  \
0               30               60               90               365   
1               25               49               66               135   
2                0                0                0                 0   
3               23               53               83               355   
4                0                0                0                 0   

   number_of_reviews  number_of_reviews_ltm  number_of_reviews_l30d  \
0                 18                      0                       0   
1                  0                      0                       0   
2                 14                     14                       1   
3                 13                     13                       3   
4                 64                     42                       3   

   review_scores_rating  review_scores_accuracy  review_scores_cleanliness  \
0              4.940000                5.000000                   4.890000   
1              4.760427                4.794096                   4.748194   
2              4.870000                5.000000                   5.000000   
3              4.080000                4.080000                   4.000000   
4              4.800000                4.740000                   4.750000   

   review_scores_chec

In [16]:
train_cols = list(train.columns)
test_cols = list(test.columns)

train_cols = [col for col in train_cols if col in test_cols]
test_cols = [col for col in test_cols if col in train_cols]

train_cols == test_cols

True

In [17]:
x_train = train[train_cols].drop(columns=['id'])
y_train = train.host_is_superhost

x_test = test[test_cols].drop(columns=['id'])

sc = StandardScaler()

sc.fit(x_train)
x_train_scaled = sc.transform(x_train)
x_test_scaled = sc.transform(x_test)

In [18]:
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform')

knn_model.fit(x_train_scaled, y_train)

y_pred = knn_model.predict(x_test_scaled)

In [21]:
# coarse grid

model = KNeighborsClassifier()


grid = {'n_neighbors':np.arange(2,1000,100), 'weights':['uniform', 'distance']}


gscv = GridSearchCV(model, grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_scaled, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict(x_test_scaled)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbors': 2, 'weights': 'distance'}
0.7886313091562229


In [25]:
# finer grid

model = KNeighborsClassifier()


grid = {'n_neighbors':np.arange(2,20,2), 'weights':['uniform', 'distance']}


gscv = GridSearchCV(model, grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_scaled, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict(x_test_scaled)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
{'n_neighbors': 2, 'weights': 'distance'}
0.7886313091562229


In [27]:
# finest grid

model = KNeighborsClassifier()


grid = {'n_neighbors':np.arange(2,20,1), 'weights':['uniform', 'distance']}


gscv = GridSearchCV(model, grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_scaled, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict(x_test_scaled)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'n_neighbors': 2, 'weights': 'distance'}
0.7886313091562229


In [30]:
best_model = gscv.best_estimator_

probas = cross_val_predict(best_model, x_train_scaled, y_train, cv=5, method='predict_proba')

thrs = np.arange(0.0, 1.001, 0.001)
best_accuracy = 0
best_thr = 0.5

for thr in thrs:
    preds = (probas[:, 1] >= thr).astype(int)
    accuracy = accuracy_score(y_train, preds)
    recall = recall_score(y_train, preds)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_thr = thr

print(f"Best Threshold: {best_thr} with Accuracy: {best_accuracy}")

probs = best_model.predict_proba(x_test_scaled)[:,1]

y_pred = (probs >= best_thr).astype(int)

Best Threshold: 0.523 with Accuracy: 0.7900341571227647


In [34]:
# Narrowing features with lasso regression

x_train = train[train_cols].drop(columns=['id'])
y_train = train.host_is_superhost

x_test = test[test_cols].drop(columns=['id'])

sc = StandardScaler()

sc.fit(x_train)
x_train_scaled = sc.transform(x_train)
x_test_scaled = sc.transform(x_test)

lasso_model = LogisticRegression(penalty='l1', solver='liblinear', C=1.0)

x_train_subset = x_train_scaled[:500, :]

lasso_model.fit(x_train_subset, y_train[:500])

coefficients = pd.DataFrame({
    'feature': x_train.columns,
    'importance': np.abs(lasso_model.coef_.flatten())
})

coefficients.sort_values(by='importance', ascending=False, inplace=True)

coefficients

feature  importance
13              maximum_minimum_nights    1.487996
30         review_scores_communication    1.069569
24               number_of_reviews_ltm    1.007648
26                review_scores_rating    0.686739
301   neighbourhood_cleansed_Lake View    0.649007
..                                 ...         ...
181  host_neighbourhood_Lincoln Square    0.000000
182    host_neighbourhood_Little India    0.000000
183    host_neighbourhood_Little Italy    0.000000
20                     availability_60    0.000000
190   host_neighbourhood_Magnolia Glen    0.000000

[381 rows x 2 columns]

In [36]:
important_features = list(coefficients.head(20).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

sc = StandardScaler()

sc.fit(x_train_reduced)
x_train_scaled = sc.transform(x_train_reduced)
x_test_scaled = sc.transform(x_test_reduced)

model = best_model

model.fit(x_train_scaled, y_train)

probs = best_model.predict_proba(x_test_scaled)[:,1]

y_pred = (probs >= best_thr).astype(int)

In [38]:
# Narrowing features with permutation importance

best_neighbors = gscv.best_params_['n_neighbors']
best_weight = gscv.best_params_['weights']

model = KNeighborsRegressor(n_neighbors=best_neighbors, weights=best_weight)
model.fit(x_train_scaled, y_train)

perm_importance = permutation_importance(model, x_train_scaled, y_train, n_repeats=10, random_state=1)

feature_names = [col for col in x_train_reduced if col != 'id' and col != 'host_is_superhost']
features_importance = pd.DataFrame({'feature': feature_names, 'importance': perm_importance.importances_mean})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

features_importance

feature  importance
15                       days_since_host_started    0.773380
2                          number_of_reviews_ltm    0.705012
7                              reviews_per_month    0.564431
3                           review_scores_rating    0.539486
5                              number_of_reviews    0.388175
0                         maximum_minimum_nights    0.370527
1                    review_scores_communication    0.289192
16  calculated_host_listings_count_private_rooms    0.277703
4               neighbourhood_cleansed_Lake View    0.131397
13           neighbourhood_cleansed_Logan Square    0.087269
6                 host_neighbourhood_River North    0.076773
14        neighbourhood_cleansed_Lower West Side    0.062509
18             host_neighbourhood_Lake View East    0.051686
17                 neighbourhood_cleansed_Uptown    0.050208
12    host_neighbourhood_South Loop/Printers Row    0.050079
11                     host_neighbourhood_Uptown    0.045375
10            host_neighbourhood_Lower West Side    0.043396
9         host_neighbourhood_West Loop/Greektown    0.037638
19                   property_type_Room in hotel    0.034400
8     host_verifications_['phone', 'work_email']    0.018090

In [39]:
important_features = list(features_importance.head(15).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

sc = StandardScaler()

sc.fit(x_train_reduced)
x_train_scaled = sc.transform(x_train_reduced)
x_test_scaled = sc.transform(x_test_reduced)

model = KNeighborsClassifier(n_neighbors=best_neighbors, weights=best_weight)

model.fit(x_train_scaled, y_train)

y_pred = model.predict(x_test_scaled)


In [41]:
# descion tree feature selection

x_train = train[train_cols].drop(columns=['id'])
y_train = train.host_is_superhost

x_test = test[test_cols].drop(columns=['id'])

model = DecisionTreeClassifier(random_state=1, max_depth=10, max_leaf_nodes=300)

model.fit(x_train, y_train)

features_importance = pd.DataFrame({'feature': x_train.columns, 'importance': model.feature_importances_})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

features_importance

feature  importance
26                          review_scores_rating    0.283116
24                         number_of_reviews_ltm    0.157579
2                            host_listings_count    0.083342
35   calculated_host_listings_count_entire_homes    0.064438
3                      host_total_listings_count    0.051885
..                                           ...         ...
150                 host_neighbourhood_Edgewater    0.000000
149                 host_neighbourhood_East Side    0.000000
148               host_neighbourhood_East Pilsen    0.000000
147            host_neighbourhood_East Hyde Park    0.000000
380                        room_type_Shared room    0.000000

[381 rows x 2 columns]

In [47]:
important_features = list(features_importance.head(20).feature)

sc = StandardScaler()

sc.fit(x_train)
x_train_scaled = sc.transform(x_train)
x_test_scaled = sc.transform(x_test)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

model = KNeighborsClassifier(n_neighbors=best_neighbors, weights=best_weight)

model.fit(x_train_reduced, y_train)

y_pred = model.predict_proba(x_test_reduced)[:,1]

y_pred = (probs >= best_thr).astype(int)

In [49]:
model = KNeighborsClassifier()


grid = {'n_neighbors':np.arange(2,20,1), 'weights':['uniform', 'distance']}

cv_settings = RepeatedKFold(random_state=1, n_repeats=5, n_splits=5)

gscv = GridSearchCV(model, grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict_proba(x_test_reduced)[:,1]

y_pred = (probs >= best_thr).astype(int)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'n_neighbors': 2, 'weights': 'distance'}
0.7627091279691631


In [50]:
ids = test.id
results = pd.DataFrame({
    'id': ids,
    'predicted': y_pred
})


results.to_csv('KNN_Classifier_submission.csv', index=False)
results.head(30)

id  predicted
0   1543972437713169913          0
1   1710552057351883447          0
2              97075525          1
3              83734823          1
4              56722823          0
5             103823601          1
6   1159028243437550345          0
7              92485765          0
8              11142401          1
9   1189875082289533291          0
10  1544820103115167001          1
11             26331127          0
12             58383089          1
13             64762469          0
14            100902043          0
15            103130147          0
16             94352581          1
17  1849015146225985937          0
18             91307779          1
19             53591385          0
20             84071033          0
21             99484949          0
22             97864023          0
23             31809543          0
24  1519081708190436577          1
25             59771473          1
26  1310888620900225001          0
27             23054201          1
28             37470529          0
29  1212015341587401973          0